# Here i make the same feature engingeering as in dataset generation, but instead of euclidean distance i take walking distance calculated by Krzysztof

In [46]:
import pandas as pd
import h3
traffic_data = pd.read_parquet("../for_participants/data_parquet/traffic_train.parquet")
pois_df = pd.read_parquet("../for_participants/data_parquet/pois.parquet")

In [47]:
wd = pd.read_parquet("../krzysztof/pois_walking_distance.parquet")

In [48]:
df_left = pd.DataFrame(wd["station_h3"].unique())
df_left.columns = ["station_h3"]

In [49]:
results = []
for dist in [0.2, 0.4, 0.6, 0.8, 1]:
    wd0 = wd[wd.walking_distance_from_station_km < dist]
    df0 = df_left.merge(wd0.groupby("station_h3")[['accommodation', 'active_life', 'arts_and_entertainment',
       'attractions_and_activities', 'automotive', 'beauty_and_spa',
       'business_to_business', 'eat_and_drink', 'education',
       'financial_service', 'health_and_medical', 'home_service', 'mass_media',
       'pets', 'private_establishments_and_corporates',
       'professional_services', 'public_service_and_government', 'real_estate',
       'religious_organization', 'retail', 'structure_and_geography', 'travel']].sum(), left_on = "station_h3", right_index = True, how = "left")
    new_columns = {
        col: col + "_wd" + str(dist) if col != "station_h3" else col
        for col in df0.columns
    }
    df0 = df0.rename(columns = new_columns)
    results.append(df0)

In [50]:
df_wd = pd.concat(results, axis =1)

In [51]:
df_wd = df_wd.fillna(0)

In [52]:
df_wd = df_wd.loc[:, ~df_wd.columns.duplicated()]

In [53]:
df_wd.to_parquet("dfs_processed/df_pois_agg_wd.parquet")

In [55]:
a = pd.read_parquet("dfs_processed/df_pois_agg.parquet")

In [61]:
import pandas as pd
import h3
traffic_data_train = pd.read_parquet("../for_participants/data_parquet/traffic_train.parquet")
traffic_data_test = pd.read_parquet("../for_participants/data_parquet/traffic_test_without_target.parquet")
pois_df = pd.read_parquet("../for_participants/data_parquet/pois.parquet")
demo_df = pd.read_parquet('../for_participants/data_parquet/demography.parquet')

In [62]:
import pandas as pd
import numpy as np

def distance_km(h1, h2):
    loc1 = h3.cell_to_latlng(h1)
    loc2 = h3.cell_to_latlng(h2)
    return h3.great_circle_distance(loc1, loc2)


def aggregate_by_dist(
    agg_points,
    data,
    dist_metric,
    allowed_distance=2,
    agg_fun=np.mean,
):
    results = []

    for point in agg_points:
        data_point = data.copy()
        distances = data_point["h3res13"].apply(lambda x: dist_metric(x, point))
        nearby = data_point[distances < allowed_distance]

        if nearby.empty:
            continue

        aggregated = nearby.drop(columns=["h3res13"]).agg(agg_fun)
        aggregated["h3res13"] = point
        results.append(aggregated)

    if not results:
        return pd.DataFrame(columns=data.columns)

    return pd.DataFrame(results)


In [81]:
import warnings
warnings.filterwarnings("ignore")
def calculate_pois(dataset):
    dfs0 = []
    for dist in [0.5, 1, 2, 3, 5, 10]:
        df0 = aggregate_by_dist(agg_points = dataset["h3res13"].unique(), data = pois_df, dist_metric = distance_km, allowed_distance = dist, agg_fun = np.sum)
        new_columns = {
            col: col + "_" + str(dist) if col != "h3res13" else col
            for col in df0.columns
        }
        df0 = df0.rename(columns = new_columns)
        dfs0.append(df0)
    df_pois_agg = pd.concat(dfs0, axis = 1)
    df_pois_agg["h3res13"] = df_pois_agg["h3res13"].bfill(axis = 1).ffill(axis = 1)
    df_pois_agg = df_pois_agg.loc[:,~df_pois_agg.columns.duplicated()]
    return df_pois_agg

In [82]:
df_pois_train = calculate_pois(dataset = traffic_data_train)

In [84]:
df_pois_train.filter(regex = "h")

,health_and_medical_0.5,home_service_0.5,private_establishments_and_corporates_0.5,structure_and_geography_0.5,h3res13,health_and_medical_1,home_service_1,private_establishments_and_corporates_1,structure_and_geography_1,health_and_medical_2,...,private_establishments_and_corporates_3,structure_and_geography_3,health_and_medical_5,home_service_5,private_establishments_and_corporates_5,structure_and_geography_5,health_and_medical_10,home_service_10,private_establishments_and_corporates_10,structure_and_geography_10
0,0.0,3.0,0.0,0.0,8d1f5224541113f,3,9,0,1,11,...,2,2,360,235,22,12,3327,1424,97,106
1,21.0,9.0,1.0,0.0,8d1f52248b302ff,51,17,1,1,193,...,5,14,921,359,21,26,2486,1096,69,79
2,10.0,5.0,1.0,0.0,8d1f5224c89b23f,24,15,2,0,78,...,9,9,846,372,21,24,3522,1536,97,117
3,28.0,5.0,0.0,0.0,8d1f5224dcdd63f,103,24,0,1,238,...,6,16,1056,426,22,30,3203,1376,93,110
4,13.0,6.0,0.0,1.0,8d1f52260d8eaff,41,24,0,1,171,...,6,7,601,363,16,14,3381,1430,99,120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,18.0,12.0,0.0,0.0,8d1f53cb3131cff,81,30,0,3,228,...,8,18,1373,600,36,68,4015,1920,106,150
66,12.0,3.0,0.0,0.0,8d1f53cb38edcff,42,22,0,1,154,...,5,16,1274,572,30,65,3966,1876,106,150
67,18.0,7.0,0.0,2.0,8d1f53cb3ce307f,92,36,0,3,310,...,8,22,1423,621,37,71,4029,1932,107,147
68,0.0,2.0,0.0,0.0,8d1f53cb6b503bf,13,6,4,1,61,...,6,14,612,306,7,25,2903,1441,85,117
